# Demo 3: Random Forest and DataStax Analytics
------
<img src="images/drinkWine.jpeg" width="300" height="500">


#### Dataset: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

## What are we trying to learn from this dataset? 

# QUESTION: Can Random Forest be used to classify a wine’s rating score by its attributes?

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas
import cassandra
import pyspark
import re
import os
import random
from random import randint, randrange
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### Helper function to have nicer formatting of Spark DataFrames

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

# DataStax Enterprise Analytics
<img src="images/dselogo.png" width="400" height="200">

## Creating Tables and Loading Tables

### Connect to DSE Analytics Cluster

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster(['dse'])
session = cluster.connect()

### Create Demo Keyspace 

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS accelerate 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

### Set keyspace 

In [6]:
session.set_keyspace('accelerate')

### Create table called `wines`. Our PRIMARY will be a unique key (wineid) we generate for each row.  This will have two datasets "white" and "red"

In [7]:
query = "CREATE TABLE IF NOT EXISTS wines \
                                   (wineid int, fixedAcidity float, volatileAcidity float, citricAcid float, sugar float, \
                                   chlorides float, freeSulfur float, totalSulfur float, density float, ph float, \
                                   sulphates float, alcohol float, quality float, \
                                   PRIMARY KEY (wineid))"
session.execute(query)

### What do these of these 12 columns represent: 

* **Fixed acidity**
* **Volatile acidity**
* **Citric Acid**
* **Residual Sugar** 
* **Chlorides**
* **Free sulfur dioxide**     
* **Total sulfur dioxide**
* **Density** 
* **pH**
* **Sulphates**
* **Alcohol**
* **Quality**

### Load 2 Wine Dataset -- White and Red
<img src="images/whiteAndRed.jpeg" width="300" height="300">

### Load Wine datasets from CSV file (winequality-red.csv winequality-white.csv)
* No clean up was requried! How nice :)

#### Insert all the Wine Data into the DSE table `wines`

In [8]:
fileName = 'data/winequality-red.csv'
input_file = open(fileName, 'r')
i = 1
for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1

fileName = 'data/winequality-white.csv'
input_file = open(fileName, 'r')

for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1
    

## Machine Learning with DSE Analytics and Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

#### Create a spark session that is connected to DSE. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [9]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()


wineDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="wines", keyspace="accelerate").load()

print ("Table Wine Row Count: ")
print (wineDF.count())

Table Wine Row Count: 
6497


In [10]:
showDF(wineDF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,5691,10.0,0.057,0.28,0.99425,6.4,21.0,3.26,6.0,7.9,0.36,82.0,0.14
1,728,9.5,0.067,0.02,0.99700,6.4,4.0,3.46,5.0,1.8,0.68,11.0,0.57
2,6490,11.8,0.036,0.29,0.98938,6.1,25.0,3.06,6.0,2.2,0.44,100.0,0.34
3,208,9.3,0.069,0.31,0.99625,7.8,26.0,3.29,5.0,1.8,0.53,120.0,0.57
4,1939,10.2,0.049,0.35,0.99340,6.6,49.0,3.43,7.0,1.5,0.85,141.0,0.18


#### Let's filter out only wines that have been rated 6.0 or higher and create a new dataframe with that information 

In [11]:
wine6DF = wineDF.filter("quality > 5")
showDF(wine6DF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,3088,9.5,0.044,0.74,0.99720,6.5,68.0,3.18,6.0,13.3,0.54,224.0,0.260
1,6395,10.7,0.035,0.29,0.99142,6.4,44.0,3.17,7.0,1.1,0.55,140.0,0.105
2,381,9.4,0.080,0.42,0.99740,8.3,11.0,3.21,6.0,2.0,0.80,27.0,0.260
3,4529,9.1,0.046,0.29,0.99834,6.6,39.0,3.05,6.0,14.4,0.50,118.0,0.220
4,4319,10.9,0.029,0.31,0.99276,7.5,53.0,3.03,6.0,6.5,0.38,160.0,0.180


#### Create Vector with all elements of the wine 

In [12]:
assembler = VectorAssembler(
    inputCols=['alcohol', 'chlorides', 'citricacid', 'density', 'fixedacidity', 'ph', 'freesulfur', 'sugar', 'sulphates', 'totalsulfur', 'volatileacidity'],
    outputCol='features')

trainingData = assembler.transform(wine6DF)

labelIndexer = StringIndexer(inputCol="quality", outputCol="label", handleInvalid='keep')
trainingData1 = labelIndexer.fit(trainingData).transform(trainingData)

showDF(trainingData1)
print(trainingData1.count())

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label
0,4317,10.8,0.046,0.29,0.99518,6.8,59.0,3.20,6.0,10.4,0.40,143.0,0.16,"[10.800000190734863, 0.04600000008940697, 0.28...",0.0
1,3372,10.9,0.059,0.26,0.99550,7.8,32.0,3.04,6.0,9.5,0.43,178.0,0.40,"[10.899999618530273, 0.05900000035762787, 0.25...",0.0
2,4830,9.4,0.056,0.57,0.99548,6.7,60.0,2.96,6.0,6.6,0.43,150.0,0.13,"[9.399999618530273, 0.0560000017285347, 0.5699...",0.0
3,2731,9.7,0.047,0.34,0.99440,6.9,24.0,3.20,6.0,4.0,0.52,128.0,0.23,"[9.699999809265137, 0.04699999839067459, 0.340...",0.0
4,769,9.7,0.082,0.02,0.99744,7.1,24.0,3.55,6.0,2.3,0.53,94.0,0.59,"[9.699999809265137, 0.0820000022649765, 0.0199...",0.0


4113


### We will be training a model with Random Forest, and because of this we need to split up our dataset in to a training and test set. Will split 80/20. 

In [13]:
# Split the data into train and test
splits = trainingData1.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

print ("Train Dataframe Row Count: ")
print (train.count())
print ("Test Datafram Row Count: ")
print (test.count())

Train Dataframe Row Count: 
3359
Test Datafram Row Count: 
752


In [14]:
!date

Wed May 15 18:10:26 PDT 2019


In [15]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

model = rf.fit(train)

predictions = model.transform(test)
#predictions.show()
print (predictions.count())
showDF(predictions)

752


,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label,rawPrediction,probability,prediction
0,9,9.5,0.073,0.02,0.9968,7.8,9.0,3.36,7.0,2.0,0.57,18.0,0.58,"[9.5, 0.0729999989271164, 0.019999999552965164...",1.0,"[9.396850311426318, 0.5508459750593168, 0.0523...","[0.9396850311426318, 0.05508459750593168, 0.00...",0.0
1,17,10.5,0.092,0.56,0.9969,8.5,35.0,3.30,7.0,1.8,0.75,103.0,0.28,"[10.5, 0.09200000017881393, 0.5600000023841858...",1.0,"[7.872448026522618, 1.81795063588903, 0.294216...","[0.7872448026522617, 0.18179506358890296, 0.02...",0.0
2,21,9.4,0.077,0.48,0.9968,8.9,29.0,3.39,6.0,1.8,0.53,60.0,0.22,"[9.399999618530273, 0.07699999958276749, 0.479...",0.0,"[9.241572488316299, 0.670204601565711, 0.07725...","[0.9241572488316301, 0.06702046015657112, 0.00...",0.0
3,109,9.6,0.091,0.53,0.9976,8.0,18.0,3.37,6.0,2.5,0.80,80.0,0.33,"[9.600000381469727, 0.09099999815225601, 0.529...",0.0,"[8.585164908248752, 1.162295200618395, 0.24652...","[0.858516490824875, 0.1162295200618395, 0.0246...",0.0
4,270,10.1,0.104,0.51,0.9996,11.5,4.0,3.28,6.0,4.0,0.97,23.0,0.18,"[10.100000381469727, 0.10400000214576721, 0.50...",0.0,"[7.155537932285342, 2.686756186951431, 0.15402...","[0.7155537932285343, 0.26867561869514317, 0.01...",0.0


In [16]:
!date

Wed May 15 18:10:30 PDT 2019


In [17]:
showDF(predictions.select("quality", "label", "prediction", "probability"))

,quality,label,prediction,probability
0,7.0,1.0,0.0,"[0.9396850311426318, 0.05508459750593168, 0.00..."
1,7.0,1.0,0.0,"[0.7872448026522617, 0.18179506358890296, 0.02..."
2,6.0,0.0,0.0,"[0.9241572488316301, 0.06702046015657112, 0.00..."
3,6.0,0.0,0.0,"[0.858516490824875, 0.1162295200618395, 0.0246..."
4,6.0,0.0,0.0,"[0.7155537932285343, 0.26867561869514317, 0.01..."


### We can now use the MutliclassClassifciationEvaluator to evalute the accurancy of our predictions. 

In [18]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7286096256684492


In [19]:
session.execute("""drop table wines""")